In [ ]:
# 필요한 라이브러리 임포트
from langchain_community.vectorstores import FAISS  # (3) 벡터 저장소 생성 라이브러리
from langchain_community.document_loaders import TextLoader  # (3) 텍스트 파일을 불러오는 로더
from langchain_text_splitters import CharacterTextSplitter  # (2) 텍스트 분할을 위한 도구
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings  # (3) 임베딩 생성기

# 1단계: 문서 로드 및 전처리
loader = TextLoader("appendix-keywords.txt", encoding="UTF-8")  # (3) 텍스트 파일을 로드합니다
documents = loader.load()  # (3) 문서를 불러옵니다

# 텍스트를 일정 크기로 나누기
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)  # (2) 텍스트를 300자 단위로 분할
split_docs = text_splitter.split_documents(documents)  # (2) 문서를 청크로 나눕니다

# 2단계: 임베딩 생성 및 벡터 저장소 구축
model_name = 'BAAI/bge-m3'  # (3) 사용하려는 HuggingFace 모델 이름
embeddings = HuggingFaceEndpointEmbeddings(model=model_name)  # (3) HuggingFace 임베딩 생성
db = FAISS.from_documents(split_docs, embeddings)  # (3) 임베딩된 문서를 FAISS 벡터 저장소에 저장

# 검색 방법별로 retriever 생성
retriever_similarity = db.as_retriever(search_type="similarity")  # (3) 유사도 기반 검색기 설정
retriever_mmr = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10, "lambda_mult": 0.5})  # (3) MMR 기반 검색기 설정
retriever_threshold = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7})  # (3) 유사도 임계값 기반 검색기 설정

# 3단계: 검색 결과를 보기 쉽게 출력하는 도우미 함수
def pretty_print_docs(docs):  # (2) 검색 결과를 형식화하여 출력하는 함수
    for i, doc in enumerate(docs):
        print(f"문서 {i+1}:\n{doc.page_content}")
        print("-" * 50)

# 4단계: 통합 질문-답변 시스템
def interactive_qa_system():  # (2) 사용자와 상호작용하는 질문-답변 시스템
    print("질문을 입력하고 검색 방식을 선택하세요:")
    query = input("질문: ")
    
    # 유사도 검색 수행
    print("\n===== 유사도 검색 결과 =====")
    docs_similarity = retriever_similarity.invoke(query)  # (3) 유사도 검색 수행
    pretty_print_docs(docs_similarity)  # (2) 결과 출력

    # MMR 검색 수행
    print("\n===== MMR 검색 결과 =====")
    docs_mmr = retriever_mmr.invoke(query)  # (3) MMR 검색 수행
    pretty_print_docs(docs_mmr)  # (2) 결과 출력

    # 유사도 임계값 기반 검색 수행
    print("\n===== 임계값 검색 결과 =====")
    docs_threshold = retriever_threshold.invoke(query)  # (3) 유사도 임계값 기반 검색 수행
    pretty_print_docs(docs_threshold)  # (2) 결과 출력

# 통합 질문-답변 시스템 실행
interactive_qa_system()


질문을 입력하고 검색 방식을 선택하세요:

===== 유사도 검색 결과 =====
문서 1:
정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation
--------------------------------------------------
문서 2:
정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
--------------------------------------------------
문서 3:
정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다. 이는 데이터 분석 작업을 효율적으로 수행할 수 있게 합니다.
예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.
연관키워드: 데이터 분석, 파이썬, 데이터 처리

GPT (Generative Pretrained Transformer)
--------------------------------------------------
문서 4:
정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.
예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.
연관키워드:

No relevant docs were retrieved using the relevance score threshold 0.5
